# Preprocessing of Crop Fire Data

To learn time series model, data is needed in such order in which model and read and process to predict future pattern of fire points in different regions

## Importing Libraries

First, All required libraries has been imported like geopandas, pandas and rasterio

In [1]:
import geopandas as gpd
import pandas as pd
import rasterio

/home/luckyw0w/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


Two types of dataset has been created. One at district level and one at mandal level(smaller region)

Mandal boundary shapefile from telangana state government has been downloaded which contains district boundary information also and fire points from NASA's FIRMS data from year 2017 to 2021 also has been downloaded. These fire points has been clipped for our area of interest which is Telangana state here

In [2]:
adm_name = gpd.read_file('data/TSDM/Mandal_Boundary.shp')  #Mandal (an administrative boundary) Shapefile
viirs1 = pd.read_csv('data/VIIRS1.csv')                    #Fire points of whole Indiafrom year 
                                                           #2017 to 2021 of VIIRS S-NPP 375m


fire_pts = gpd.GeoDataFrame(viirs1,                        #Converting fire points into dataframe
                         geometry=gpd.points_from_xy(
                             viirs1.longitude,
                             viirs1.latitude),
                         crs=4326)

fire_telangana = fire_pts.clip(adm_name)                   #Clipping fire points with Telangana boundaries

fire_telangana = fire_telangana.sort_values(by=["acq_date"])#Sorting points by date

In [3]:
fire_telangana.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,geometry
106,18.608213,79.866440,342.32,0.46,0.47,2017-01-01,724,N,VIIRS,n,1,301.80,5.88,D,0,POINT (79.86644 18.60821)
306,18.472372,80.036087,329.72,0.46,0.47,2017-01-01,724,N,VIIRS,l,1,301.18,4.64,D,0,POINT (80.03609 18.47237)
205,18.360744,79.849739,337.62,0.47,0.48,2017-01-01,724,N,VIIRS,n,1,301.93,4.23,D,0,POINT (79.84974 18.36074)
297,17.310617,78.753777,341.89,0.58,0.52,2017-01-01,724,N,VIIRS,n,1,304.53,10.34,D,0,POINT (78.75378 17.31062)
296,17.310413,78.757889,335.14,0.58,0.52,2017-01-01,724,N,VIIRS,n,1,303.49,5.90,D,0,POINT (78.75789 17.31041)


To classify each fire points into crop fire, values of fire points has been taken from the Land Use Land Cover (LULC) map of ESRI (downloaded from [DiCRA](https://dicra.undp.org.in/) platform). Only LULC of 2021 used for all years crop class for the ease of the code.

In [4]:
#list of coordinates for all points
coords = [(x,y) for x, y in zip(fire_telangana.longitude, fire_telangana.latitude)]

# Open the raster and store metadata
src = rasterio.open('data/TelanganaLULC_2021.tif')

# Sample the raster at every point location and store values in GeoDataFrame
fire_telangana['Class'] = [x[0] for x in src.sample(coords)]

#Creating dataframe with of crop fire(class = 5)
crop_fire = fire_telangana.loc[fire_telangana ['Class'] == 5]
del crop_fire['Class']  # deleteing class column

In [5]:
crop_fire.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,geometry
106,18.608213,79.866440,342.32,0.46,0.47,2017-01-01,724,N,VIIRS,n,1,301.80,5.88,D,0,POINT (79.86644 18.60821)
306,18.472372,80.036087,329.72,0.46,0.47,2017-01-01,724,N,VIIRS,l,1,301.18,4.64,D,0,POINT (80.03609 18.47237)
205,18.360744,79.849739,337.62,0.47,0.48,2017-01-01,724,N,VIIRS,n,1,301.93,4.23,D,0,POINT (79.84974 18.36074)
105,18.615726,79.572044,332.22,0.49,0.48,2017-01-01,724,N,VIIRS,n,1,301.81,2.85,D,0,POINT (79.57204 18.61573)
299,17.488455,78.201614,331.08,0.62,0.54,2017-01-01,724,N,VIIRS,l,1,302.31,3.47,D,0,POINT (78.20161 17.48845)


Joining both dataframes of boundaries and fire points, and counting fire events in each mandal for each date. For this, a list of date created and fire events have been counted for each dates with columns of unique administrative boundary names.

In [6]:
#joining adm boundary dataframe and fir epoint dataframe to get location of each fire points
df = gpd.sjoin(crop_fire, adm_name, how="inner", op='intersects')
df = df.reset_index()
del df['index']

/home/luckyw0w/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3191: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [7]:
#creating new dataframe which consists mandal names, dates and fire counts
df1 = pd.DataFrame()
df1['acq_date']=df['acq_date']
df1['Mandal_Nam']=df['Mandal_Nam']
df1['fireCount']=1

#suming the occurence of fire by grouping mandal name and dates
df1 = df1.groupby(['acq_date','Mandal_Nam']).sum().reset_index()

In [8]:
#Creating the dataframe for each boundary with number of fire counts on each dates  
dflist = []
val = pd.DataFrame()
for i in df1['Mandal_Nam'].unique():
    val =  df1.loc[df1['Mandal_Nam'] == i]
    val['Mandal_Nam']=val['fireCount']
    val.rename(columns = {'Mandal_Nam': i }, inplace = True)
    del val['fireCount']
    dflist.append(val)
datelist = []
for d in pd.date_range(start="2017-01-01",end="2021-12-31"):
  datelist.append(str(d.strftime("%Y-%m-%d")))
date = pd.DataFrame()
date['acq_date']= datelist
dflist.append(date)
df2 = pd.concat(dflist, axis=0)
df2 = df2.reset_index()
del df2['index']
df2 = df2.fillna(0).groupby(['acq_date']).sum().reset_index()

/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Mandal_Nam']=val['fireCount']
/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Nam': i }, inplace = True)
/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Mandal_Nam']=val['fireCount']
/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Nam': i }, inplace = True)
/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Mandal_Nam']=val['fireCount']
/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Nam': i }, inplace = True)
/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Mandal_Nam']=val['fireCount']
/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Nam': i }, inplace = True)
/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Nam': i }, inplace = True)
/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Mandal_Nam']=val['fireCount']
/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Na

...

/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Mandal_Nam']=val['fireCount']
/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Nam': i }, inplace = True)
/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Mandal_Nam']=val['fireCount']
/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Nam': i }, inplace = True)
/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Mandal_Nam']=val['fireCount']
/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Nam': i }, inplace = True)
/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Mandal_Nam']=val['fireCount']
/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Nam': i }, inplace = True)
/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Nam': i }, inplace = True)
/tmp/ipykernel_1374874/2976768868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Mandal_Nam']=val['fireCount']
/tmp/ipykernel_1374874/2976768868.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Mandal_Na

Merge all the columns into one which gives dataframe with columns like date,place,fireCounts. Dump dataframe into csv for further analysis:

In [9]:
#merging all columns into one
dflist = []
for column in df2:
    val = pd.DataFrame()
    val["Date"]=df2['acq_date']
    val['place']=column
    val['fireCount']= df2[column]
    dflist.append(val)
df = pd.DataFrame()
df = pd.concat(dflist[1:], axis=0)
df = df.reset_index()
del df['index']

df.to_csv('data/cropfire_mandal.csv', index = False)

Some mandals has been removed because of no crop fire events happened in that region. That could be the case of urban areas majority mandals. These on crop fire event mandals can be displayed as below:

In [10]:
leftarea = []
#Mandal boundaries which doesnt have any crop fire points 
for i in adm_name.Mandal_Nam.unique():
    if i not in list(df.place.unique()):
        leftarea.append(i)

Similarly, dataset can be created for district level crop fire events also: 

In [11]:
#Similarly for districts
adm_name = gpd.read_file('data/TSDM/Mandal_Boundary.shp')  #Mandal (an administrative boundary) Shapefile
viirs1 = pd.read_csv('data/VIIRS1.csv')                    #Fire points of whole Indiafrom year 
                                                           #2017 to 2021 of VIIRS S-NPP 375m


fire_pts = gpd.GeoDataFrame(viirs1,                        #Converting fire points into dataframe
                         geometry=gpd.points_from_xy(
                             viirs1.longitude,
                             viirs1.latitude),
                         crs=4326)

fire_telangana = fire_pts.clip(adm_name)                   #Clipping fire points with Telangana boundaries

fire_telangana = fire_telangana.sort_values(by=["acq_date"])#Sorting points by date


#list of coordinates for all points
coords = [(x,y) for x, y in zip(fire_telangana.longitude, fire_telangana.latitude)]

# Open the raster and store metadata
src = rasterio.open('data/TelanganaLULC_2021.tif')

# Sample the raster at every point location and store values in GeoDataFrame
fire_telangana['Class'] = [x[0] for x in src.sample(coords)]

#Creating dataframe with of crop fire(class = 5)
crop_fire = fire_telangana.loc[fire_telangana ['Class'] == 5]
del crop_fire['Class']  # deleteing class column

#joining adm boundary dataframe and fir epoint dataframe to get location of each fire points
df = gpd.sjoin(crop_fire, adm_name, how="inner", op='intersects')
df = df.reset_index()
del df['index']

#creating new dataframe which consists district names, dates and fire counts
df1 = pd.DataFrame()
df1['acq_date']=df['acq_date']
df1['Dist_Name']=df['Dist_Name']
df1['fireCount']=1

#suming the occurence of fire by grouping district names and dates
df1 = df1.groupby(['acq_date','Dist_Name']).sum().reset_index()

#Creating the dataframe for each boundary with number of fire counts on each dates 
dflist = []
val = pd.DataFrame()
for i in df1['Dist_Name'].unique():
    val =  df1.loc[df1['Dist_Name'] == i]
    val['Dist_Name']=val['fireCount']
    val.rename(columns = {'Dist_Name': i }, inplace = True)
    del val['fireCount']
    dflist.append(val)
datelist = []
for d in pd.date_range(start="2017-01-01",end="2021-12-31"):
  datelist.append(str(d.strftime("%Y-%m-%d")))
date = pd.DataFrame()
date['acq_date']= datelist
dflist.append(date)
df2 = pd.concat(dflist, axis=0)
df2 = df2.reset_index()
del df2['index']
df2 = df2.fillna(0).groupby(['acq_date']).sum().reset_index()

#merging all columns into one
dflist = []
for column in df2:
    val = pd.DataFrame()
    val["Date"]=df2['acq_date']
    val['place']=column
    val['fireCount']= df2[column]
    dflist.append(val)
df = pd.DataFrame()
df = pd.concat(dflist[1:], axis=0)
df = df.reset_index()
del df['index']

df.to_csv('data/cropfire_dist.csv', index = False)

/home/luckyw0w/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3191: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_1374874/2876724651.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Dist_Name']=val['fireCount']
/tmp/ipykernel_1374874/2876724651.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Dist_Name': i }, inplace = True)
/tmp/ipykernel_1374874/287672

/tmp/ipykernel_1374874/2876724651.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['Dist_Name']=val['fireCount']
/tmp/ipykernel_1374874/2876724651.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.rename(columns = {'Dist_Name': i }, inplace = True)
/tmp/ipykernel_1374874/2876724651.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

Urban district which doesnt have any firer events can see displayed as below:

In [12]:
#Districts boundaries which doesnt have any crop fire points 
for i in adm_name.Dist_Name.unique():
    if i not in list(df.place.unique()):
        print(i)
        

Hyderabad


Similarly, dataset can be created for district level crop fire events also: 

In [ ]:
#Similarly for districts
adm_name = gpd.read_file('data/TSDM/Mandal_Boundary.shp')  #Mandal (an administrative boundary) Shapefile
viirs1 = pd.read_csv('data/VIIRS1.csv')                    #Fire points of whole Indiafrom year 
                                                           #2017 to 2021 of VIIRS S-NPP 375m


fire_pts = gpd.GeoDataFrame(viirs1,                        #Converting fire points into dataframe
                         geometry=gpd.points_from_xy(
                             viirs1.longitude,
                             viirs1.latitude),
                         crs=4326)

fire_telangana = fire_pts.clip(adm_name)                   #Clipping fire points with Telangana boundaries

fire_telangana = fire_telangana.sort_values(by=["acq_date"])#Sorting points by date


#list of coordinates for all points
coords = [(x,y) for x, y in zip(fire_telangana.longitude, fire_telangana.latitude)]

# Open the raster and store metadata
src = rasterio.open('data/TelanganaLULC_2021.tif')

# Sample the raster at every point location and store values in GeoDataFrame
fire_telangana['Class'] = [x[0] for x in src.sample(coords)]

#Creating dataframe with of crop fire(class = 5)
crop_fire = fire_telangana.loc[fire_telangana ['Class'] == 5]
del crop_fire['Class']  # deleteing class column

#joining adm boundary dataframe and fir epoint dataframe to get location of each fire points
df = gpd.sjoin(crop_fire, adm_name, how="inner", op='intersects')
df = df.reset_index()
del df['index']

#creating new dataframe which consists district names, dates and fire counts
df1 = pd.DataFrame()
df1['acq_date']=df['acq_date']
df1['N_Revenue']=df['N_Revenue']
df1['fireCount']=1

#suming the occurence of fire by grouping district names and dates
df1 = df1.groupby(['acq_date','N_Revenue']).sum().reset_index()

#Creating the dataframe for each boundary with number of fire counts on each dates 
dflist = []
val = pd.DataFrame()
for i in df1['N_Revenue'].unique():
    val =  df1.loc[df1['N_Revenue'] == i]
    val['N_Revenue']=val['fireCount']
    val.rename(columns = {'N_Revenue': i }, inplace = True)
    del val['fireCount']
    dflist.append(val)
datelist = []
for d in pd.date_range(start="2017-01-01",end="2021-12-31"):
  datelist.append(str(d.strftime("%Y-%m-%d")))
date = pd.DataFrame()
date['acq_date']= datelist
dflist.append(date)
df2 = pd.concat(dflist, axis=0)
df2 = df2.reset_index()
del df2['index']
df2 = df2.fillna(0).groupby(['acq_date']).sum().reset_index()

#merging all columns into one
dflist = []
for column in df2:
    val = pd.DataFrame()
    val["Date"]=df2['acq_date']
    val['place']=column
    val['fireCount']= df2[column]
    dflist.append(val)
df = pd.DataFrame()
df = pd.concat(dflist[1:], axis=0)
df = df.reset_index()
del df['index']

df.to_csv('data/cropfire_taluk.csv', index = False)

Urban Taluks/Tehsils which doesnt have any firer events can see displayed as below:

In [ ]:
#Taluk/Tehsil boundaries which doesnt have any crop fire points 
for i in adm_name.N_Revenue.unique():
    if i not in list(df.place.unique()):
        print(i)